# Technical note: OpenCL simulation

Multi-cellular models in Myokit use:

\begin{equation}
\frac{\partial V}{\partial t}=-\frac{1}{C_{m}}(i_{\mathit{ion}}+i_{\mathit{stim}}+i_{\mathit{diff}})\label{eq:dv/dt}
\end{equation}

where $i_\mathit{diff}$ is a diffusion current defined as

\begin{equation}
i_{\mathit{diff},i}=\sum_{j\in N_{i}}g_{ij}\left(V_{i}-V_{j}\right)\label{eq:idiff}
\end{equation}

and 

- $N_{i}$ is the set of neighbours of cell $i$;
- $g_{ij}$ is the conductance between cell $i$ and its neighbour $j$.

In the current simulation classes, only two conductances ($g_{x}$ and $g_{y}$) can be specified and all cells are assumed to be in a regular grid.
This means all central cells have two x-neighbours and two y-neighbours, while cells at the boundaries have 2 or 3 neighbouring cells.
In other words:

\begin{equation}
i_{\mathit{diff},i}=g_{x}\sum_{x\in N_{x}}\left(V_{i}-V_{x}\right)+g_{y}\sum_{y\in N_{y}}\left(V_{i}-V_{y}\right)
\end{equation}

where $N_{x}$ and $N_{y}$ are the set of neighbours in the $x$ and $y$ direction respectively.

## Relationship to mono-domain equations

The bidomain model starts from the assumption of ohmic conductance currents as in the equation above.
It then adds a few components representing the extracellular conductance and generalises the result to a continuous PDE.
The mono-domain is derived from the bi-domain model by dropping the terms explicitly modelling the extracellular conductance.
Many modelers then assume only conduction in the principal directions (x,y and z) and add the boundary condition that no current enters or leaves the system at the sides.
Then, using a finite-difference approximation of the second order derivatives arising from the bidomain equations,
we arrive right back at the original equation specifying ohmic conductance between neighbouring cells.

While the PDE form is highly advantageous when discretising to an arbitrary-shaped mesh, for simple cell-to-cell conductance models it has the disadvantage of introducing extra parameters (with perhaps a less clear meaning than $g_{x}$ and $g_{y}$).
However, in order to compare results in Myokit with mono-domain simulations, it may be necessary to translate beteen the used parameter sets.
Their relationship is derived below.

According to the mono-domain model, each point in the heart can be described as:

\begin{equation}
\frac{\lambda}{1+\lambda}\nabla\cdot\left(\Sigma_{i}\nabla v\right)=\chi\left(C_{m}\frac{\partial v}{\partial t}+i_{\mathit{ion}}+i_{\mathit{stim}}\right)\label{eq:monodom}
\end{equation}

With:

- $\lambda$ The intra- to extracellular conductivity ratio
- $\Sigma_{i}$ A tensor representing the intracellular conductivity
- $v$ The local membrane potential
- $\chi$ The surface area of the membrane per unit volume
- $C_{m}$ Capacitance per unit area
- $i_{ion}$ Ionic current per unit area

Assuming all currents are normalised to unit area, we can combine this with the equation for $\frac{\partial V}{\partial t}$ to find the mono-domain equation for $i_{diff}$:

\begin{eqnarray*}
\frac{\lambda}{1+\lambda}\nabla\cdot\left(\Sigma_{i}\nabla V\right) & = & \chi\left(C_{m}\frac{\partial V}{\partial t}+i_{\mathit{ion}}+i_{\mathit{stim}}\right)\\
\frac{\lambda}{\chi\left(1+\lambda\right)}\nabla\cdot\left(\Sigma_{i}\nabla V\right) & = & C_{m}\frac{\partial V}{\partial t}+i_{\mathit{ion}}+i_{\mathit{stim}}\\
\frac{\partial V}{\partial t} & = & \frac{-1}{C_{m}}\left(i_{\mathit{ion}}+i_{\mathit{stim}}-\frac{1}{\chi}\frac{\lambda}{\left(1+\lambda\right)}\nabla\cdot\left(\Sigma_{i}\nabla V\right)\right)\\
 & = & \frac{-1}{C_{m}}\left(i_{\mathit{ion}}+i_{\mathit{stim}}+i_{\mathit{diff}}\right)
\end{eqnarray*}

so that

\begin{equation}
i_{\mathit{diff}}=-\frac{1}{\chi}\frac{\lambda}{\left(1+\lambda\right)}\nabla\cdot\left(\Sigma_{i}\nabla V\right)
\end{equation}

For the 2D case, let

\begin{equation}
\Sigma_{i}=\sigma_{x}\hat{\mathbf{i}}+\sigma_{y}\hat{\mathbf{j}}
\end{equation}

then

\begin{equation}
\Sigma_{i}\nabla V=\frac{\partial V}{\partial x}\sigma_{x}\hat{\mathbf{i}}+\frac{\partial V}{\partial y}\sigma_{y}\hat{\mathbf{j}}
\end{equation}

and

\begin{equation}
\nabla\cdot\left(\Sigma_{i}\nabla V\right)=\frac{\partial^{2}V}{\partial x^{2}}\sigma_{x}+\frac{\partial^{2}V}{\partial y^{2}}\sigma_{y}
\end{equation}

A finite difference approximation of $\frac{\partial^{2}V}{\partial x^{2}}$ is

\begin{eqnarray*}
\frac{\partial^{2}V}{\partial x^{2}} & = & \frac{V_{i-1}-2V_{i}+V_{i+1}}{\Delta x^{2}}\\
 & = & -\left(\frac{V_{i}-V_{i-1}}{\Delta x^{2}}+\frac{V_{i}-V_{i+1}}{\Delta x^{2}}\right)\\
 & = & \frac{-1}{\Delta x^{2}}\sum_{j\in N_{x}}\left(V_{i}-V_{x}\right)
\end{eqnarray*}

where $N_{x}$ is the set of neighbours of cell $i$ in the $x$ direction.
Using this approximation, we find:

\begin{eqnarray*}
i_{\mathit{diff}} & = & \frac{\sigma_{x}}{\chi\Delta x^{2}}\frac{\lambda}{1+\lambda}\sum_{j\in N_{x}}\left(V_{i}-V_{x}\right)+\frac{\sigma_{y}}{\chi\Delta y^{2}}\frac{\lambda}{1+\lambda}\sum_{j\in N_{y}}\left(V_{i}-V_{y}\right)\\
 & = & g_{x}\sum_{x\in N_{x}}\left(V_{i}-V_{x}\right)+g_{y}\sum_{y\in N_{y}}\left(V_{i}-V_{y}\right)\\
 & = & \sum g_{ij}\left(V_{i}-V_{j}\right)
\end{eqnarray*}

So to convert the four mono-domain parameters to a single conductance, we use:
\begin{eqnarray*}
g_{x} & = & \frac{\lambda\sigma_{x}}{\chi\left(1+\lambda\right)\Delta x^{2}}\\
g_{y} & = & \frac{\lambda\sigma_{y}}{\chi\left(1+\lambda\right)\Delta y^{2}}
\end{eqnarray*}